In [1]:
from openai import OpenAI
import dotenv
import rich
dotenv.load_dotenv('/Users/shepardxia/Desktop/directory/commonsense/.env')

True

In [3]:
import torch
from torch import nn
import pandas as pd
import mrf
import mrf_data
import json


In [4]:
import llm
import json

In [15]:
initial_message = [{"role": "system", "content": 'You are a statistician interviewing for a market analytics position. ' +\
                    'During the interview, you are asked to analyze the nightly prices of Airbnb listings in the United States in 2017. ' +\
                    'Although you do not have access to any actual data, you are known for your ability to identify structures and relationships among data variables. ' +\
                    'Rely on commonsense guesses, intuition, and experience to make broad generalizations. ' +\
                    'The interviewer will guide your thinking through a set of predefined questions. ' +\
                    'Aim to be thoughtful but also general and concise. ' +\
                    'Reason through the problem concisely before giving a structured answer in the end of your response. ' +\
                    'You should not give additional justifications after giving the answers.'},]

# put comments in JSON
formatq = 'Now, I need you to translate the variables and constraints (leave constraints empty if not discussed yet) discussed so far into JSON. Do not forget about price. Follow the following example as format, if there are two variables, a with values a1, a2, a3, and b with values b1, b2. \nProbability of a1 given b2 is 10%~20%, of a2 given b1 is 50%, and marginal probability of a1 (i.e. not conditioned on any variable) is 20%~30%. This would translate into \n {\n  "variables": {\n    "a": ["a1", "a2", "a3"],\n    "b": ["b1", "b2"]\n  },\n  "constraints": [\n    [["b", "b2"], ["a", "a1"], [0.1, 0.2]],\n    [["b", "b1"], ["a", "a2"], [0.5]],\n    [null, ["a", "a1"], [0.2, 0.3]]\n  ]\n}'

# start with coarse gram facts bi->tri

In [7]:
import importlib
importlib.reload(llm)
#agent = llm.LLM(initial_message, model='gpt-3.5-turbo', json_format=formatq)
agent = llm.LLM(initial_message, json_format=formatq)
agent.reset()

In [15]:
binq = 'What kind of statistical distribution do you think the nightly price of Airbnb listings most resembles? Divide the price into a few ranges accordingly.'

brainq = "Now let's brainstorm. What aspects do you think affect or determine the prices? How would they interact with each other? They would be variables we would like to model."

moreq = "Good, now try to reason through more variables and interactions."

varq = "Pick some most important variables, what exhaustively are their possible values? These values should be general and few. \n Then also include the price as a variable in the end."

margq = "Well, what are the marginal probabilities of all the variables? Give ranges that capture your uncertainty."


In [16]:
binq = 'To model the nightly price of Airbnb listings in 2017, we need to first divide the price into a few ranges. Think about the parametric statistical distribution to approximate the price. Divide the price into a few ranges accordingly.'

brainq = "Let's brainstorm the factors that might affect or determine the prices of Airbnb listings. '+\
    'Think about the obvious and commonsense aspects that would influence pricing. '+\
        'Reason through how these variables might interact or correlate with each other. "

broadq = "What are some salient interactions? Consider how these variables might influence each other, rather than just their impact on price. Think about correlations and dependencies among the variables themselves."

#moreq = "Take your time to reason through additional variables and their possible interactions. Consider other factors that might influence the price. Once you have thought it through, share your ideas on these additional variables."

varq = "Pick 5 most important variables, what exhaustively are their possible values? These values should be few but also concrete, and mutually exclusive. In the end, also include the price, with the ranges you gave."

# p(x)
margq = "Well, what are the individual marginal probabilities of all the values of these variables? You can give a credible interval if unsure."

# p(price|bedroom)
pricecondq = 'How would they affect the price? Suggest the probability of price, conditioned on each of these variable values. Give ranges that capture your uncertainty.'

# p(bedroom=2|property_type=entire_house)
#interactq = "How do you think the rest of the chosen variables interact with each other? Avoid duplicates from before. After reasoning through, suggest the conditional probability that captures those interaction."
# dependencies <-> interaction? point-wise mutual information 
interactq = "What are some dependencies among these variables other than price? Suggest the conditional probability that captures those interaction."

acoq = "Good, suggest a couple more."

#Think and reason about the most important variables that affect the price, and have meaningful interactions among themselves. Then in the end, p
chains = [binq, brainq, broadq, varq]
saws = [None, margq, pricecondq, interactq]



In [17]:
print(agent.button(chains, saws))

user : To model the nightly price of Airbnb listings in 2017, we need to first the price into a few ranges. Think about the parametric statistical distribution to approximate the price. Divide the price into a few ranges accordingly.


--------------------------------------------------------------------------------------


assistant : To model the nightly prices of Airbnb listings, we can consider a parametric statistical distribution that is often used for modeling prices and other positive-valued data: the log-normal distribution. The log-normal distribution is appropriate because it can model a wide range of values with a positive skew, which is common in price data.

Given this, we can divide the nightly prices into the following ranges:

1. **Low Range**: $0 - $50
   - This range captures budget accommodations such as shared rooms, hostels, or listings in less expensive areas.

2. **Mid-Low Range**: $51 - $150
   - This range includes more affordable private rooms or entire homes 

In [21]:
vars = agent.var_json
constraints_list = agent.cons_json
print(vars)

{'Geographic Location': ['Urban', 'Suburban', 'Rural', 'Tourist Destination'], 'Property Type': ['Entire Home/Apartment', 'Private Room', 'Shared Room', 'Unique Property'], 'Seasonality': ['Peak Season', 'Off-Peak Season', 'Shoulder Season'], 'Amenities': ['Basic', 'Moderate', 'Luxury'], 'Booking Lead Time': ['Last-Minute', 'Short-Term', 'Medium-Term', 'Long-Term'], 'Price': ['Low Range', 'Mid-Low Range', 'Mid Range', 'Mid-High Range', 'High Range']}


In [22]:
constraints_list

[[None, ['Geographic Location', 'Urban'], [0.4, 0.5]],
 [None, ['Geographic Location', 'Suburban'], [0.2, 0.3]],
 [None, ['Geographic Location', 'Rural'], [0.1, 0.15]],
 [None, ['Geographic Location', 'Tourist Destination'], [0.15, 0.25]],
 [None, ['Property Type', 'Entire Home/Apartment'], [0.5, 0.6]],
 [None, ['Property Type', 'Private Room'], [0.25, 0.35]],
 [None, ['Property Type', 'Shared Room'], [0.05, 0.1]],
 [None, ['Property Type', 'Unique Property'], [0.05, 0.1]],
 [None, ['Seasonality', 'Peak Season'], [0.3, 0.4]],
 [None, ['Seasonality', 'Off-Peak Season'], [0.3, 0.4]],
 [None, ['Seasonality', 'Shoulder Season'], [0.2, 0.3]],
 [None, ['Amenities', 'Basic'], [0.3, 0.4]],
 [None, ['Amenities', 'Moderate'], [0.4, 0.5]],
 [None, ['Amenities', 'Luxury'], [0.1, 0.2]],
 [None, ['Booking Lead Time', 'Last-Minute'], [0.1, 0.2]],
 [None, ['Booking Lead Time', 'Short-Term'], [0.2, 0.3]],
 [None, ['Booking Lead Time', 'Medium-Term'], [0.3, 0.4]],
 [None, ['Booking Lead Time', 'Long-Ter

In [31]:
print([constraints_list])

[[[None, ['Geographic Location', 'Urban'], [0.4, 0.5]], [None, ['Geographic Location', 'Suburban'], [0.2, 0.3]], [None, ['Geographic Location', 'Rural'], [0.1, 0.15]], [None, ['Geographic Location', 'Tourist Destination'], [0.15, 0.25]], [None, ['Property Type', 'Entire Home/Apartment'], [0.5, 0.6]], [None, ['Property Type', 'Private Room'], [0.25, 0.35]], [None, ['Property Type', 'Shared Room'], [0.05, 0.1]], [None, ['Property Type', 'Unique Property'], [0.05, 0.1]], [None, ['Seasonality', 'Peak Season'], [0.3, 0.4]], [None, ['Seasonality', 'Off-Peak Season'], [0.3, 0.4]], [None, ['Seasonality', 'Shoulder Season'], [0.2, 0.3]], [None, ['Amenities', 'Basic'], [0.3, 0.4]], [None, ['Amenities', 'Moderate'], [0.4, 0.5]], [None, ['Amenities', 'Luxury'], [0.1, 0.2]], [None, ['Booking Lead Time', 'Last-Minute'], [0.1, 0.2]], [None, ['Booking Lead Time', 'Short-Term'], [0.2, 0.3]], [None, ['Booking Lead Time', 'Medium-Term'], [0.3, 0.4]], [None, ['Booking Lead Time', 'Long-Term'], [0.1, 0.2]]

In [32]:
print([vars])

[{'Geographic Location': ['Urban', 'Suburban', 'Rural', 'Tourist Destination'], 'Property Type': ['Entire Home/Apartment', 'Private Room', 'Shared Room', 'Unique Property'], 'Seasonality': ['Peak Season', 'Off-Peak Season', 'Shoulder Season'], 'Amenities': ['Basic', 'Moderate', 'Luxury'], 'Booking Lead Time': ['Last-Minute', 'Short-Term', 'Medium-Term', 'Long-Term'], 'Price': ['Low Range', 'Mid-Low Range', 'Mid Range', 'Mid-High Range', 'High Range']}]


In [56]:
from importlib import reload
reload(mrf)
reload(mrf_data)

# guidance package?
test = mrf.Brute()
test.from_json(vars=vars, constraints=constraints_list)
test.update(1000)


-----------------------------------------------
Finished building
Geographic Location ['Urban', 'Suburban', 'Rural', 'Tourist Destination']
Property Type ['Entire Home/Apartment', 'Private Room', 'Shared Room', 'Unique Property']
Seasonality ['Peak Season', 'Off-Peak Season', 'Shoulder Season']
Amenities ['Basic', 'Moderate', 'Luxury']
Booking Lead Time ['Last-Minute', 'Short-Term', 'Medium-Term', 'Long-Term']
Price ['Low Range', 'Mid-Low Range', 'Mid Range', 'Mid-High Range', 'High Range']
tensor([4, 4, 3, 3, 4, 5], dtype=torch.int32)
Features:  125
Constraints:  124
-----------------------------------------------


  0%|          | 2/1000 [00:00<01:06, 15.11it/s]

Loss:  tensor(2.1042, grad_fn=<AddBackward0>)
Violated:  63  out of  112


  3%|▎         | 34/1000 [00:01<00:34, 27.71it/s]


KeyboardInterrupt: 

In [35]:
print(test.nvars())
print(test.nfeatures())



6
125


In [50]:
print(test.var_name2ind('Seasonality'))
print(test.var_ind2name(2))
print(test.val_name2ind('Seasonality', 'Off-Peak Season'))
print(test.val_ind2name(2, 1))

2
Seasonality
2
Peak Season


In [72]:
di = dict()
di['Seasonality'] = ['Off-Peak Season']
di[0] = [1]
test.marg('Geographic Location', di)


tensor([1.0000, 1.2268, 0.7484, 0.3165])

In [71]:
test.infer_print(target=0, conditions=['Off-Peak Season', 'Urban'])

Conditions:  ['Off-Peak Season', 'Urban']
                            0
Urban                1.000000
Suburban             1.226847
Rural                0.748417
Tourist Destination  0.316489


[1.0, 1.2268474102020264, 0.7484167814254761, 0.31648921966552734]

In [46]:
test.variable_lookup

{'Geographic Location': 0,
 0: 'Geographic Location',
 'Urban': (0, 1.0),
 (0, 1): 'Urban',
 'Suburban': (0, 2.0),
 (0, 2): 'Suburban',
 'Rural': (0, 3.0),
 (0, 3): 'Rural',
 'Tourist Destination': (0, 4.0),
 (0, 4): 'Tourist Destination',
 'Property Type': 1,
 1: 'Property Type',
 'Entire Home/Apartment': (1, 1.0),
 (1, 1): 'Entire Home/Apartment',
 'Private Room': (1, 2.0),
 (1, 2): 'Private Room',
 'Shared Room': (1, 3.0),
 (1, 3): 'Shared Room',
 'Unique Property': (1, 4.0),
 (1, 4): 'Unique Property',
 'Seasonality': 2,
 2: 'Seasonality',
 'Peak Season': (2, 1.0),
 (2, 1): 'Peak Season',
 'Off-Peak Season': (2, 2.0),
 (2, 2): 'Off-Peak Season',
 'Shoulder Season': (2, 3.0),
 (2, 3): 'Shoulder Season',
 'Amenities': 3,
 3: 'Amenities',
 'Basic': (3, 1.0),
 (3, 1): 'Basic',
 'Moderate': (3, 2.0),
 (3, 2): 'Moderate',
 'Luxury': (3, 3.0),
 (3, 3): 'Luxury',
 'Booking Lead Time': 4,
 4: 'Booking Lead Time',
 'Last-Minute': (4, 1.0),
 (4, 1): 'Last-Minute',
 'Short-Term': (4, 2.0),
 (4

In [24]:
#test.set_w0(-1)
test.update(1000)

  0%|          | 4/1000 [00:00<00:48, 20.71it/s]

Loss:  tensor(2.1042, grad_fn=<AddBackward0>)
Violated:  63  out of  112


 11%|█         | 107/1000 [00:03<00:28, 30.87it/s]

Loss:  tensor(0.0214, grad_fn=<AddBackward0>)
Violated:  34  out of  112


 21%|██        | 206/1000 [00:07<00:24, 31.83it/s]

...

 80%|████████  | 804/1000 [00:27<00:07, 27.92it/s]

Loss:  tensor(0.0081, grad_fn=<AddBackward0>)
Violated:  42  out of  112


 90%|█████████ | 902/1000 [00:30<00:03, 31.22it/s]

Loss:  tensor(0.0079, grad_fn=<AddBackward0>)
Violated:  37  out of  112


100%|██████████| 1000/1000 [00:34<00:00, 29.32it/s]


In [25]:
test.variable_names()

['Geographic Location', 'Property Type', 'Seasonality', 'Amenities', 'Booking Lead Time', 'Price']


In [26]:
test.vars_print()


Geographic Location ['Urban', 'Suburban', 'Rural', 'Tourist Destination']
Property Type ['Entire Home/Apartment', 'Private Room', 'Shared Room', 'Unique Property']
Seasonality ['Peak Season', 'Off-Peak Season', 'Shoulder Season']
Amenities ['Basic', 'Moderate', 'Luxury']
Booking Lead Time ['Last-Minute', 'Short-Term', 'Medium-Term', 'Long-Term']
Price ['Low Range', 'Mid-Low Range', 'Mid Range', 'Mid-High Range', 'High Range']
tensor([4, 4, 3, 3, 4, 5], dtype=torch.int32)


In [27]:
ret = test.infer_print()

Conditions:  None
                       0
Low Range       0.137715
Mid-Low Range   0.324761
Mid Range       0.309258
Mid-High Range  0.156726
High Range      0.071541


In [30]:
ret = test.infer_print(conditions=['Entire Home/Apartment', 'Urban'])

Conditions:  ['Entire Home/Apartment', 'Urban']
                       0
Low Range       0.013354
Mid-Low Range   0.122281
Mid Range       0.287852
Mid-High Range  0.332170
High Range      0.244343


In [99]:
ret = test.infer_print()

Conditions:  None
                     0
$0 - $50      0.156319
$50 - $100    0.242188
$100 - $200   0.288569
$200 - $500   0.191271
$500 - $1000  0.087047
$1000+        0.034605


In [116]:
ret = test.infer_print(conditions=['2 Bedrooms', 'Urban'])

Conditions:  ['2 Bedrooms', 'Urban']
                     0
$0 - $50      0.078429
$50 - $100    0.158255
$100 - $200   0.333385
$200 - $500   0.251176
$500 - $1000  0.123529
$1000+        0.055226


In [115]:
ret = test.infer_print(conditions=['2 Bedrooms', 'Urban', 'Peak Season', 'Luxury', 'Entire Home/Apartment'])


Conditions:  ['2 Bedrooms', 'Urban', 'Peak Season', 'Luxury', 'Entire Home/Apartment']
                     0
$0 - $50      0.003279
$50 - $100    0.022148
$100 - $200   0.314232
$200 - $500   0.281583
$500 - $1000  0.239166
$1000+        0.139592


In [121]:
cond = lambda x: x[x['bedrooms'] == 2]
dat.marg(cond=cond)

-1 - 50: 	0.92%
50 - 100: 	10.96%
100 - 200: 	46.99%
200 - 500: 	36.36%
500 - 1000: 	3.97%
1000 - 1999: 	0.80%


In [133]:
from importlib import reload

reload(mrf_data)
dat = mrf_data.data()
dat.set_bins([50, 100, 200, 500, 1000])

All columns available:  Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds',
       'price'],
      dtype='object')
Columns used:  ['log_price', 'bathrooms', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'amenities', 'review_scores_rating', 'city']


In [134]:
dat.unique('city')

['NYC' 'SF' 'DC' 'LA' 'Chicago' 'Boston']


In [124]:
dat.describe('property_type')

count         74019
unique           35
top       Apartment
freq          48929
Name: property_type, dtype: object


In [129]:
dat.marg('property_type')

Apartment: 	66.10%
House: 	22.29%
Condominium: 	3.59%
Loft: 	1.68%
Townhouse: 	2.29%
Hostel: 	0.09%
Guest suite: 	0.17%
Bed & Breakfast: 	0.62%
Bungalow: 	0.49%
Guesthouse: 	0.67%
Dorm: 	0.19%
Other: 	0.82%
Camper/RV: 	0.13%
Villa: 	0.24%
Boutique hotel: 	0.09%
Timeshare: 	0.10%
In-law: 	0.10%
Boat: 	0.09%
Serviced apartment: 	0.03%
Castle: 	0.02%
Cabin: 	0.10%
Treehouse: 	0.01%
Tipi: 	0.00%
Vacation home: 	0.01%
Tent: 	0.02%
Hut: 	0.01%
Casa particular: 	0.00%
Chalet: 	0.01%
Yurt: 	0.01%
Earth House: 	0.01%
Parking Space: 	0.00%
Train: 	0.00%
Cave: 	0.00%
Lighthouse: 	0.00%
Island: 	0.00%


In [52]:
messages = agent.iter_chat(new_messages=chains)

user : To model the nightly price of Airbnb listings in 2017, we need to first the price into a few ranges. Think about the parametric statistical distribution to approximate the price. Divide the price into a few ranges accordingly.


--------------------------------------------------------------------------------------


assistant : Considering the wide range of nightly prices for Airbnb listings, we can divide the prices into four ranges: 
1. Low prices (e.g., $0-$50)
2. Medium-low prices (e.g., $51-$100)
3. Medium-high prices (e.g., $101-$200)
4. High prices (e.g., $201 and above)

For modeling the nightly prices, we can consider using a skewed distribution like the log-normal distribution. This distribution is commonly used for modeling positive-valued variables that are skewed to the right, which could be suitable for the distribution of nightly prices for Airbnb listings.


--------------------------------------------------------------------------------------


user : Let's brai

In [12]:
agent.reset()

In [45]:
agent.initial_message

[{'role': 'system',
  'content': 'You are a statistician interviewing for a market analytics position. During the interview, you are asked to analyze the nightly prices of Airbnb listings in the United States in 2017. Although you do not have access to any actual data, you are known for your ability to identify structures and relationships among data variables. Rely on commonsense guesses, intuition, and experience to make broad generalizations. The interviewer will guide your thinking through a set of predefined questions. Aim to be thoughtful but also general and conside. Reason through the problem considely before giving a structured answer in the end of your response. You should not give additional justifications after giving the answers.'}]

In [77]:
messages = agent.continue_from_last(new_user_message=acoq)

user : Good, suggest a couple more.


--------------------------------------------------------------------------------------


assistant : Another potential dependency among the variables is the interaction between **Property Type** and **Seasonality**. Certain property types may be more popular or in demand during specific seasons.

Conditional probability based on this dependency:
- Given Entire Home/Apartment:
  - Probability of Peak Season: 40%
  - Probability of Shoulder Season: 30%
  - Probability of Low Season: 30%

- Given Private Room:
  - Probability of Peak Season: 30%
  - Probability of Shoulder Season: 40%
  - Probability of Low Season: 30%

- Given Shared Room:
  - Probability of Peak Season: 20%
  - Probability of Shoulder Season: 30%
  - Probability of Low Season: 50%

This conditional probability captures how the type of property may interact with seasonality, reflecting potential shifts in demand for different property types during various seasons.

Another dependency

In [67]:
agent.current_dialogue

[{'role': 'system',
  'content': 'You are a statistician interviewing for a market analytics position. During the interview, you are asked to analyze the nightly prices of Airbnb listings in the United States in 2017. Although you do not have access to any actual data, you are known for your ability to identify structures and relationships among data variables. Rely on commonsense guesses, intuition, and experience to make broad generalizations. The interviewer will guide your thinking through a set of predefined questions. Aim to be thoughtful but also general and conside. Reason through the problem considely before giving a structured answer in the end of your response. You should not give additional justifications after giving the answers.'},
 {'role': 'user',
  'content': 'To model the nightly price of Airbnb listings in 2017, we need to first the price into a few ranges. Think about the parametric statistical distribution to approximate the price. Divide the price into a few range

In [78]:
messages = agent.get_json()

user : Now, I need you to translate the variables and constraints (leave constraints empty if not discussed yet) discussed so far into JSON. Do not forget about price. Follow the following example as format, if there are two variables, a with values a1, a2, a3, and b with values b1, b2. 
Probability of a1 given b2 is 10%~20%, of a2 given b1 is 50%, and marginal probability of a1 (i.e. not conditioned on any variable) is 20%~30%. This would translate into 
 {
  "variables": {
    "a": ["a1", "a2", "a3"],
    "b": ["b1", "b2"]
  },
  "probabilities": [
    [["b", "b2"], ["a", "a1"], [0.1, 0.2]],
    [["b", "b1"], ["a", "a2"], [0.5]],
    [null, ["a", "a1"], [0.2, 0.3]]
  ]
}
We already have the following json:
{
  "variables": {
    "Location": ["Urban", "Suburban", "Rural"],
    "Property Type": ["Entire Home/Apartment", "Private Room", "Shared Room"],
    "Accommodation Size": ["Small", "Medium", "Large"],
    "Amenities": ["Basic", "Standard", "Premium"],
    "Seasonality": ["Low Seas

In [79]:
agent.prent(agent.current_dialogue)

system : You are a statistician interviewing for a market analytics position. During the interview, you are asked to analyze the nightly prices of Airbnb listings in the United States in 2017. Although you do not have access to any actual data, you are known for your ability to identify structures and relationships among data variables. Rely on commonsense guesses, intuition, and experience to make broad generalizations. The interviewer will guide your thinking through a set of predefined questions. Aim to be thoughtful but also general and conside. Reason through the problem considely before giving a structured answer in the end of your response. You should not give additional justifications after giving the answers.


--------------------------------------------------------------------------------------


user : To model the nightly price of Airbnb listings in 2017, we need to first the price into a few ranges. Think about the parametric statistical distribution to approximate the pr

In [33]:
print([agent.current_json])

['{\n  "variables": {\n    "Location": ["Urban", "Suburban", "Rural"],\n    "Property Type": ["Apartment", "House", "Villa"],\n    "Seasonality": ["Peak season", "Off-peak season"],\n    "Size and Capacity": ["Small (1-2 guests)", "Medium (3-6 guests)", "Large (7+ guests)"],\n    "Reviews and Ratings": ["Low ratings", "Average ratings", "High ratings"],\n    "Price Range": ["Low ($0 - $50)", "Medium ($51 - $150)", "High ($151 - $300)", "Luxury ($301+)"]\n  },\n  "probabilities": [\n    [null, "Location", [0.4, 0.6, 0.3, 0.5, 0.1, 0.3]],\n    [null, "Property Type", [0.4, 0.6, 0.3, 0.5, 0.1, 0.3]],\n    [null, "Seasonality", [0.4, 0.6, 0.3, 0.5]],\n    [null, "Size and Capacity", [0.3, 0.5, 0.4, 0.6, 0.1, 0.3]],\n    [null, "Reviews and Ratings", [0.2, 0.4, 0.4, 0.6, 0.2, 0.4]]\n  ]\n}']


In [71]:
ret = agent.backspace()

system : You are a statistician interviewing for a market analytics position. During the interview, you are asked to analyze the nightly prices of Airbnb listings in the United States in 2017. Although you do not have access to any actual data, you are known for your ability to identify structures and relationships among data variables. Rely on commonsense guesses, intuition, and experience to make broad generalizations. The interviewer will guide your thinking through a set of predefined questions. Aim to be thoughtful but also general and conside. Reason through the problem considely before giving a structured answer in the end of your response. You should not give additional justifications after giving the answers.


--------------------------------------------------------------------------------------


user : To model the nightly price of Airbnb listings in 2017, we need to first the price into a few ranges. Think about the parametric statistical distribution to approximate the pr

In [199]:
agent.current_dialogue

In [48]:
updated_messages = agent.continue_from_last(margq)

user : Well, what are the marginal probabilities of all the variables? You can give a small range whose width reflect your confidence.


--------------------------------------------------------------------------------------


assistant : 1. Location:
   - Urban: 0.4 - 0.6
   - Suburban: 0.3 - 0.5
   - Rural: 0.1 - 0.3

2. Property Type:
   - Apartment: 0.4 - 0.6
   - House: 0.3 - 0.5
   - Condo: 0.1 - 0.3

3. Size of the Property:
   - Small: 0.3 - 0.5
   - Medium: 0.3 - 0.5
   - Large: 0.1 - 0.3

4. Amenities Offered:
   - Basic: 0.4 - 0.6
   - Standard: 0.3 - 0.5
   - Luxury: 0.1 - 0.3

5. Seasonality:
   - Peak: 0.4 - 0.6
   - Off-Peak: 0.3 - 0.5

Price Range:
   - Low: 0.3 - 0.5
   - Medium: 0.3 - 0.5
   - High: 0.1 - 0.3


--------------------------------------------------------------------------------------




In [137]:
print([assistant_response])

['\n{\n  "variables": {\n    "LocationType": ["Urban", "Suburban", "Rural"],\n    "PropertyType": ["EntireHomeApartment", "PrivateRoom", "SharedRoom", "UniqueProperty"],\n    "SizeCapacity": ["Small", "Medium", "Large"],\n    "Amenities": ["Basic", "Standard", "Luxury"],\n    "Seasonality": ["PeakSeason", "OffPeakSeason"],\n    "HostReputation": ["NewHost", "AverageHost", "HighlyRatedHost", "Superhost"],\n    "DemandLocalEvents": ["NormalDemand", "HighDemand"],\n    "LengthOfStay": ["ShortStay", "MediumStay", "LongStay"],\n    "Price": ["EconomyRange", "StandardRange", "PremiumRange", "LuxuryRange", "UltraLuxuryRange"]\n  },\n  "probabilities": [\n    [null, ["LocationType", "Urban"], [0.4, 0.5]],\n    [null, ["LocationType", "Suburban"], [0.3, 0.4]],\n    [null, ["LocationType", "Rural"], [0.2, 0.3]],\n    \n    [null, ["PropertyType", "EntireHomeApartment"], [0.5, 0.6]],\n    [null, ["PropertyType", "PrivateRoom"], [0.3, 0.4]],\n    [null, ["PropertyType", "SharedRoom"], [0.05, 0.1]]